<h1><a href="https://arxiv.org/abs/1710.03875">
Learning Task Specifications from Demonstrations</a></h1>
by Marcell Vazquez-Chanlatte et al.


<h2>Summary</h2>
* Infer the posterior distribution of task specification from demonstration by using Bayesian theory
* Use the MaxEnt framework and utilize the convexity of KL-divergence to facilitate the analysis.
* Treat the specification as an ascending chain and solve the problem by using spanning tree algorithm

<h2>Motivation</h2>

Some robotics tasks are non-Markovian, for instance, when the agent must retrieve the history of previous steps to make next decision.
* One can encode temporal variation into state. But then the state space will blow up in size.

Some robotics tasks are compositional in that there are subtasks
* Using quantitative rewards to indicate the subtasks is known to produce unpredictable result.
* Traditional IRL provides no principled mechanism for composing the resulting rewards. 
    * No guarantee on what kind of rewards it infers from human demonstration. Optimality w.r.t. the demonstration may not be enough
    * Using product of two automata cannot effectively solve the problem of history retrieving but add up numerous states.

<h2>Background</h2>

The environments are modeled by `MDP` $M$.
<li>State space is $S$.</li>
<li>Action space is $A$.</li>
<li>Transition function is $\delta:S\times A\times S \rightarrow [0, 1] $.</li>
<li>Reward function is $R: S\rightarrow \mathbb{R}$.</li> 

Given a specification $\varphi$ and a fixed trace length $\tau\in\mathbb{N}$, the specifcation can be represented as a set of acceptable traces $\varphi:(S\times A)^\tau\rightarrow\{0,1\}$.

Given a trace $\xi$, if $\xi$ is accepted by $\varphi$, then $\varphi(\xi)=1$. Otherwise, $\varphi(\xi)=0$. 
<li>$true(\xi)=1$</li>
<li>$false(\xi)=1$</li>
<li>$\neg\varphi=1-\varphi$</li>
<li>$[\varphi_1\wedge\varphi_2]:\xi\rightarrow min(\varphi_1(\xi), \varphi_2(\xi))$</li>
<li>$[\varphi_1\vee\varphi_2]:\xi\rightarrow max(\varphi_1(\xi), \varphi_2(\xi))$</li>
* Since $\varphi$ is binary
    * $max[\varphi_1\wedge\varphi_2]=max(min(\varphi_1, \varphi_2))=1$ simply means satisfying both $\varphi_1$ and $\varphi_2$
    * $max[\varphi_1\vee\varphi_2]=max(\varphi_1, \varphi_2)=1$ means satisfying at least one of $\varphi_1$ and $\varphi_2$
    * $max[\neg\varphi_1\vee\varphi_2]=max(\neg\varphi_1, \varphi_2)=1$ means that satisfying $\varphi_1$ should result in satisfying $\varphi_2$(implication).



<h3>Specification Inference from Demonstrations</h3>


Assumes that the set of demonstrations is $X$ in which the elements are individual paths $\xi$. The specification, in the form of set of paths $\Phi$, is a subset of all paths with length $\tau$, i.e. $\Phi\subset 2^(S\times A)^\tau$. The probability of human demonstrations following specification $\varphi$ is 
$$\varphi^*\in argmax_{\varphi\in\Phi} P(\varphi|M,X)$$

As $P(\varphi|M,X)=\frac{P(X|\varphi, M)P(\varphi|M)}{P(X|M)}$, the problem becomes solving $P(X|\varphi, M),P(\varphi|M)$ and $P(X|M)$.

In the framework of MaxEnt, for a path $\xi$, given the accumulated path feature vector $f(\xi)$ and parameter vector $\theta$ such that the path reward is $R(\xi)=\theta^T f(\xi)$, the probability of generating this path $\xi$ is 
$$P(\xi|M)=\frac{e^{\theta^T f(\xi)}}{Z}\prod^{t=\tau-1}_{t=0}\sigma(s_{t+1},s_t, a_t)$$

Now change the Markovian reward setting by replace the path accumulated reward $\theta^T f(\xi)$ into boolean $\lambda_\varphi \varphi(\xi)$. Then the path distribution becomes
$$P(\xi|M,\varphi)=\frac{e^{\lambda\varphi(\xi)}}{Z}\prod^{t=\tau-1}_{t=0}\sigma(s_{t+1},s_t, a_t)$$
where $\lambda$ and $Z$ must satisfy $\sum_\xi P(\xi|M,\varphi)=1$.


Define specification satisfaction rate in $X$ as. $\bar{\\\varphi}\triangleq\mathbb{E}_{\xi\sim X}[\varphi]=\sum_{\xi\in 2^{(S\times A)^\tau}}P(\xi|M,\varphi)\varphi(\xi)\approx \frac{\sum_{\xi\in X}\varphi(\xi)}{|X|}$. Define specification satisfaction rate for random action as $\hat{\\\varphi}\triangleq\mathbb{E}_{\xi\sim M}[\varphi]=\sum_{\xi\in 2^{(S\times A)^\tau}}P(\xi|M)\varphi(\xi)\approx \frac{\sum_{\xi\in X}\varphi(\xi)}{|X|}$

Then $P(\xi|M, \varphi)=P(\xi|M)\cdot
\begin{array}{cc}
  \{ & 
    \begin{array}{cc}
      \bar{\\\varphi}/\hat{\\\varphi} & \xi\in\varphi \\
      \bar{\\\neg\varphi}/\hat{\\\neg\varphi} & \xi\notin\varphi
    \end{array}
\end{array}
$
> The proof is in the appendix of the paper. **Note that, according to the derivation, this equation only holds when $\varphi$ is defined to be boolean. If $\varphi$ evaluates the robustness of paths as in STL, then this equation does not hold.**

Define $N_\varphi$ as the number of paths in $X$ that satisfy $\varphi(\xi)=1$ while $N_{\neg\varphi}$ that satisfy $\varphi(\xi)=0$. Then 

$$ln(P(X|M,\varphi)=ln(\prod_{\xi\in X} P(\xi|M,\varphi))=ln\prod_{\xi\in X} P(\xi|M) + N_\varphi ln(\bar{\\\varphi}/\hat{\\\varphi})+N_{\neg\varphi}ln(\bar{\\\neg\varphi}/\hat{\\\neg\varphi})$$
$$=ln(\rho(X))+N_\varphi ln(\bar{\\\varphi}/\hat{\\\varphi})+N_{\neg\varphi}ln(\bar{\\\neg\varphi}/\hat{\\\neg\varphi})$$
where $$N_\varphi=\sum_X \varphi(x)\approx |X|\cdot \bar{\\\varphi}$$
$$N_\varphi=\sum_X \neg\varphi(x)\approx |X|\cdot \bar{\\\neg\varphi}$$
Insert two Bernoulli distributions that respectively have mean $\bar{\\\varphi}$ and $\hat{\\\varphi}$ to the equation above. Then
$$ln(P(X|M,\varphi))\approx ln(\rho(X))+|X|\cdot D_{KL}(B(\bar{\\\varphi})||B(\hat{\\\varphi}))$$
>**KL-divergence can be interpolated here only because of the Boolean property of $\varphi$.**

Note that due to the Bernoulli distributions, $P(X|M,\varphi)=P(X|M,\neg\varphi)$. Therefore, $P(\varphi|M)$ must introduce bias to $\varphi$ so that satisfying the specification would be prefered. In this paper, beta distribution is adopted such that
$$P(\bar{\\\varphi}=\theta|M,\varphi, \alpha,\beta)=Beta(\theta;\alpha,\beta)\propto\theta^{\alpha-1}(1-\theta)^{\beta-1}$$
where $\alpha>3$ and $\beta\approx 1$.

In the end, the posterior distribution of $\varphi$ is 
$$ln(P(\varphi|M,X,\alpha,\beta)\approx ln(\rho(X))+|X|\cdot D_{KL}(B(\bar{\\\varphi}||B(\hat{\\\varphi})+ln(Beta(\bar{\\\varphi};\alpha,\beta))$$
>**As a matter of fact, I think $ln(\rho(X))$ should be eliminated in the equation, whereas this does not affect the theories and conclusion of the paper.**

<h2>Maximize the Posterior Distribution</h2>


Define $\Phi_i\triangleq {\varphi\in\Phi|N_\varphi=i}$ and $f_i(x)\triangleq ln(P(\varphi|M,X,\alpha,\beta, \bar{\\\varphi}=\frac{i}{|X|}, \hat{\\\varphi}=x))$.

**$\bar{\\\varphi}$ is monotonous w.r.t. $\varphi$. Because $\forall\varphi',\varphi\in\Phi$, if $\varphi'\subset\varphi$, then $\hat{\\\varphi'}\in\hat{\\\varphi}$.** 

The specifications in $\Phi_i$ can form at least one ascending sequences, or `ascending chains`, $\{\varphi_k\}$ where $\varphi_k\subset\varphi_{k+1}$. The entire $\Phi_i$ can be covered by at most $|\Phi_i|$ ascending chains. 

For each individual chain $A$ in $\Phi_i$, **because KL-divergence is convex w.r.t $\hat{\\\varphi}$,** then
$$max_{\varphi\in A}(f_i(\hat{\\\varphi}))=max(f_i(\hat{\\\varphi_i}), f_i(\hat{\\\varphi_{|A|}}))$$

In each $\Phi_i$, a maximal ascending chain $A$ can be found so that $\forall\varphi\in\Phi_i\backslash A, \exists \varphi'\in A, \varphi'\not\subset \varphi \wedge \varphi\not\subset\varphi'$. 


<h3>When there is only one maximal ascending chain $A$ in $\Phi$</h3>

Define $A_i=\{\varphi\in A|N_{\varphi}=i\}$ while the head and tail of chain $A_i$ are $A_i.\top$ and $A_i.\bot$. Then to find the specification with maximal posterior distribution is to solve 
$$max_{i\in\{0,1,\ldots,|X|}(max_{\varphi\in A_i}(f_i(\hat{\\\varphi})))=max_{i\in\{0,1,\ldots,|X|}(max(f_i(A_i.\bot), f_i(A_i.\top)))$$

The paper uses binary search to find the bottom and top for each $A_i\subset A$ with $i\in\{0, 1, \ldots, |X|\}$. The final optimality is solved by comparing all the candidate solutions.

<h3>When there are more than one maximal ascending chains $\{A\}$ in $\Phi$</h3> 

With a pair of terminal nodes $\bot, \top$ added to $\Phi$ such that all maximal chains start from $\bot$ and end in $\top$, a spanning tree is used to represent the chains.

Solving optimality for each chain one by one and comparing among the solutions is brutal and expensive. 

As the chains may share subsequences and there is no reason to repeatedly consider the nodes in the shared subsequences, the paper use DFS to find out the deepest chain in the tree and use the same method in one maximal chain case to solve optimality along this longest chain. Afterwards, the longest chain is eliminated from the tree. The algorithm iteratively searches for the longest chain in the remaining of the tree and solves optimality from the longest chain. Still, the final optimality is solved by comparing among the optimal solutions in all the iterations.

> **Why not find maximal chain(s) in every $\Phi_i$? There is no analysis in this paper on whether it is enough to only consider the maximal chain(s) in $\Phi$.**

<h3>Opinion</h3>

Infering specification from observation by using a generative model and maximal likelihood method is inspiring. I think the method used in this paper can be combined with the Bayesian Optimization method used in paper <a href="https://arxiv.org/abs/1802.08678">Verifying Controllers Against Adversarial Examples with Bayesian Optimization</a>. 

However, the theories in this paper rely on the setting that the satisfaction of the specification for each path has to be Boolean. 

This limitation results in a gap between the two papers, as the Baysesian Optimization based on Gaussian process model requires the satisfaction to be quantifiable.

However, the gap can be closed by considering the posterior distribution of specification, rather than the satisfaction of the specification.

My primitive thought is as follows to falsify a controller by inferring an environment where the controller is prone to fail.

1. Parameterize environments with $\theta\in\Theta$. Define an universal MDP $M$ that covers all possible environments, e.g. only one MDP is explicitly learnt by an NN model despite the scenario may change. Define a hyperproperty $\Phi:\Theta\rightarrow (S\times A)^\tau$ for the controller such that in any arbitrary environment $\theta$, the controller should satisfy property $\Phi(\theta)$. Supposed that the controller has already learnt a policy $\pi$ and the MDP $M$ becomes DTMC $D$.

2. Given an environment $\theta$ and the set of samples $X(\theta)$ that the controller generates in this environment. Define likelihood function $f(\theta)=max_{\varphi(\theta)\subset\Phi(\theta)} ⁡ln⁡[P[\varphi(\theta)|D, X(\theta)]$ to infer the maximal likelihood that the controller satisfy the specification in the environment. Use a similar method in this paper to solve $f(\theta)$.
> Use maximum because the controller may satisfy a subset of $\Phi(\theta)$ with a larger probability than that of $\Phi(\theta)$ as $\Phi(\theta)$ may contains large number of redudant paths. Satisfying the subset of $\Phi(\theta)$ implies satisfying $\Phi(\theta)$.

3. Use GP optimization to model $\theta∼f(\theta)$ from history $\{\theta_k,f(\theta_k)\}_1^{n−1}$, where each historical data point is solved in step 2.

3. Solve $min_\theta⁡[f(\theta)]$ by using optimization methods to find the environment $\theta$ that tends to minimize the likelihood of satisfying the hyperproperty $\Phi$. 

The algorithm ends when time out, or satisfaction rate for some environment $\theta$ is lower than some threshold.

